# Fitting a FLMP
In this notebook, we will fit a Fuzzy Logical Model of Perception (FLMP) to audiovisual speech perception data.

In [10]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.stats import binom
from scipy.special import softmax
from scipy.optimize import minimize, LinearConstraint, Bounds

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [16]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
data_A = data[:, 0, :]
data_V = data[:, 1, :]
data_AV = data[:, 2:, :]

## Fit FLMP

In [12]:
# create softmax function
# def softmax():
#     raise NotImplementedError

In [13]:
def objective_function(theta, data, n_samples=24):
    # extract audio and visual parameters
    theta_A = theta[0:K]
    theta_V = theta[K: ]

    # get probabilities for audio and visual
    p_A = softmax(theta_A).reshape(-1,1)
    p_V = softmax(theta_V).reshape(-1,1)
    
    # compute audiovisual probabilities by
    # taking the outer product for all combinations of audio and visual
    p_AV = (p_A @ p_V.T) / (p_A @ p_V.T + (1 - p_A) @ (1 - p_V).T)

    # compute the log-likelihoods
    L_A = binom.logpmf(data[0], n_samples, p_A ).flatten()
    L_V = binom.logpmf(data[1], n_samples, p_V ).flatten()
    L_AV = binom.logpmf(data[2:], n_samples, p_AV).flatten()
    L = np.concatenate([L_A, L_V, L_AV]).sum()

    # return the negative log-likelihood
    return -L

In [29]:
def flmp_fit(subject_number):
    theta = np.zeros(K*2)
    opt_result = minimize(objective_function, theta, args=(data[subject_number], n_samples))
    objective, theta_A, theta_V, success = (
        opt_result.fun, 
        (opt_result.x[0:K]), 
        (opt_result.x[K:]), 
        opt_result.success
    )
    return objective, theta_A, theta_V

In [49]:
# perform FLMP fit for each subject
theta_A = np.zeros((K, N))
theta_V = np.zeros((K, N))
neg_L = np.zeros(K)
for i in range(K):
    obj, tA, tV = flmp_fit(i)

    # print results
    print(f'\nResults for subject {i+1}')
    print(f'Negative log-likelihood: {obj}')
    print("Audio:", tA)
    print("Visual:", tV)
    neg_L[i] = obj
    theta_A[i, :] = tA
    theta_V[i, :] = tV


Results for subject 1
Negative log-likelihood: 1991.0090793646566
Audio: [-0.56082418 -0.12026456  0.16164677  0.24711243  0.27215999]
Visual: [-0.27722709 -0.12429275 -0.02976641  0.1950368   0.23619663]

Results for subject 2
Negative log-likelihood: 2163.470630921111
Audio: [-0.72308405 -0.06943153  0.17484697  0.29822473  0.32073657]
Visual: [-0.23186772 -0.05925499  0.02818901  0.09825619  0.16623447]

Results for subject 3
Negative log-likelihood: 1851.3917708039003
Audio: [-0.30291732 -0.11840685  0.06439409  0.15778014  0.19893428]
Visual: [-0.35488703 -0.23139429 -0.01874918  0.26417184  0.34061777]

Results for subject 4
Negative log-likelihood: 1269.7717884082258
Audio: [-0.20312541 -0.09689638 -0.02194963  0.08420121  0.23749587]
Visual: [-0.5434168  -0.42640156 -0.24808346  0.51627723  0.70137618]

Results for subject 5
Negative log-likelihood: 1764.099522657972
Audio: [-0.25725804 -0.05059468  0.02294307  0.11314843  0.17098669]
Visual: [-0.51011694 -0.39250776  0.007038